<a href="https://colab.research.google.com/github/nanafish/ORS/blob/main/%E5%88%B0%E9%81%94%E5%9C%8F%E5%88%86%E6%9E%90%E3%83%95%E3%82%A9%E3%83%BC%E3%83%9E%E3%83%83%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ============================================
# 任意会場版（APIキー・会場名・座標を入力）
# 非円形アイソクローン(15/30/45分) → GeoJSON/KML
# ドーナツ(非重複リング)KML → Zip化 → 自動DL（Colab）
# ============================================

!pip -q install openrouteservice shapely geopandas fiona simplekml pyproj

# ---- 入力（実行時に貼り付け/入力）----
ORS_API_KEY = input("ORS_API_KEY を貼り付けて Enter: ").strip()
VENUE_NAME  = input("会場名（例: チトセピア）: ").strip()
coord_raw   = input("中心座標（Googleマップの「緯度, 経度」例: 32.7881, 129.8562）: ").strip()

# ---- プリセット（必要ならここを変更）----
PROFILE = "driving-car"               # "driving-car" / "foot-walking" / "cycling-regular"
RANGES_MIN = [15, 30, 45]             # 1次/2次/3次（分）
SMOOTH_TOLERANCE = 0.0002             # Shapely簡素化（軽量化/視認性）
SMOOTHING_API = 5                     # ORS smoothing(0-100)
AUTO_DOWNLOAD = True                  # Zipを自動DL

# ---- 実処理 ----
import os, re, shutil
from datetime import datetime
import geopandas as gpd
from shapely.geometry import MultiPolygon
from shapely import union_all
import openrouteservice as ors
import simplekml
from google.colab import files

def _assert(cond, msg):
    if not cond:
        raise ValueError(msg)

# 検証
_assert(bool(ORS_API_KEY) and ORS_API_KEY != "YOUR_ORS_API_KEY", "ORS_API_KEY を設定してください。")
_assert(len(VENUE_NAME) > 0, "会場名を入力してください。")

# 会場名をファイル用スラグに
def slugify(s: str) -> str:
    # 日本語OKだが、危ない文字はアンダースコアに
    return re.sub(r'[\\/:*?"<>|]+', '_', s).strip()

VENUE_SLUG = slugify(VENUE_NAME)

# 座標パース（緯度, 経度）
def parse_latlon(s: str):
    s = s.replace("，", ",")  # 全角コンマ対策
    m = re.match(r"^\s*([+-]?\d+(\.\d+)?)\s*,\s*([+-]?\d+(\.\d+)?)\s*$", s)
    _assert(m is not None, "座標は「緯度, 経度」で入力してください（例: 32.7881, 129.8562）。")
    lat = float(m.group(1)); lon = float(m.group(3))
    _assert(-90 <= lat <= 90 and -180 <= lon <= 180, "座標の範囲が不正です。")
    return [lat, lon]

CENTER_COORD = parse_latlon(coord_raw)

print("Key preview:", ORS_API_KEY[:4] + "..." + ORS_API_KEY[-4:])
print(f"会場名: {VENUE_NAME} / 座標: lat={CENTER_COORD[0]:.6f}, lon={CENTER_COORD[1]:.6f}")

# ORS 呼び出し
client = ors.Client(key=ORS_API_KEY)
center_lonlat = [CENTER_COORD[1], CENTER_COORD[0]]  # ORSは [lon, lat]

try:
    res = client.isochrones(
        locations=[center_lonlat],
        profile=PROFILE,
        range=[m * 60 for m in RANGES_MIN],
        range_type="time",
        attributes=["area"],
        smoothing=SMOOTHING_API
    )
except Exception as e:
    code = getattr(e, "status", None) or getattr(e, "status_code", None)
    msg  = getattr(e, "message", None) or str(e)
    raise RuntimeError(
        "ORS API エラー: " + (f"HTTP {code} " if code else "") + msg +
        "\n403の主因: キーでIsochrones未許可 / ドメイン・IP制限 / キー無効化 / プレースホルダー使用"
    ) from e

features = res.get("features", [])
_assert(features, "アイソクローン取得に失敗（featuresが空）。座標・APIキー・PROFILEを確認。")

# GeoDataFrame化（小→大：15→30→45）
features_sorted = sorted(features, key=lambda f: f["properties"]["value"])
gdf = gpd.GeoDataFrame.from_features(features_sorted, crs="EPSG:4326")
gdf["geometry"] = gdf["geometry"].simplify(SMOOTH_TOLERANCE, preserve_topology=True)

# 出力フォルダ（会場名スラグ入り）
ts = datetime.now().strftime("%Y%m%d_%H%M")
base = f"{VENUE_SLUG}_非円形到達圏_{PROFILE}_{ts}"
os.makedirs(base, exist_ok=True)

# 保存関数
geojson_paths, kml_paths = [], []
def save_layer(sub_gdf, layer_name):
    gj_path = os.path.join(base, f"{base}_{layer_name}.geojson")
    sub_gdf.to_file(gj_path, driver="GeoJSON", encoding="utf-8")
    geojson_paths.append(gj_path)
    kml = simplekml.Kml()
    geom = sub_gdf.iloc[0].geometry
    polys = list(geom.geoms) if isinstance(geom, MultiPolygon) else [geom]
    folder = kml.newfolder(name=layer_name)
    for poly in polys:
        coords = list(poly.exterior.coords)
        pol = folder.newpolygon(name=layer_name, outerboundaryis=coords)
        pol.style.polystyle.fill = 1
        pol.style.polystyle.outline = 1
    kml_path = os.path.join(base, f"{base}_{layer_name}.kml")
    kml.save(kml_path)
    kml_paths.append(kml_path)

# 1次/2次/3次 保存（ラベルは会場汎用）
for i, minutes in enumerate(RANGES_MIN, start=1):
    layer_name = f"{i}次_" + (f"車{minutes}分" if PROFILE=="driving-car" else f"{PROFILE}_{minutes}分")
    save_layer(gdf.iloc[[i-1]].copy(), layer_name)

# 中心点KML（会場名でラベル）
kmlp = simplekml.Kml()
p = kmlp.newpoint(name=VENUE_NAME, coords=[(center_lonlat[0], center_lonlat[1])])
p.style.labelstyle.scale = 1.2
p.description = f"{VENUE_NAME}（基点）"
center_kml = os.path.join(base, f"{base}_中心点.kml")
kmlp.save(center_kml)

# ドーナツ版KML
donuts_dir = os.path.join(base, "donuts_v2")
os.makedirs(donuts_dir, exist_ok=True)
g1, g2, g3 = gdf.iloc[[0]].copy(), gdf.iloc[[1]].copy(), gdf.iloc[[2]].copy()
inner15, inner30 = union_all(list(g1.geometry.values)), union_all(list(g2.geometry.values))
ring1 = g1.copy()
ring2 = g2.copy(); ring2["geometry"] = ring2.geometry.difference(inner15)
ring3 = g3.copy(); ring3["geometry"] = ring3.geometry.difference(inner30)

def to_kml(gdf_in, out_path, layer_name):
    kml = simplekml.Kml()
    for geom in gdf_in.geometry:
        if geom.is_empty: continue
        polys = list(geom.geoms) if isinstance(geom, MultiPolygon) else [geom]
        folder = kml.newfolder(name=layer_name)
        for poly in polys:
            coords = list(poly.exterior.coords)
            pol = folder.newpolygon(name=layer_name, outerboundaryis=coords)
            pol.style.polystyle.fill = 1
            pol.style.polystyle.outline = 1
    kml.save(out_path)

to_kml(ring1, os.path.join(donuts_dir, "1次_車15分_ドーナツ.kml"), "1次_車15分")
to_kml(ring2, os.path.join(donuts_dir, "2次_車30分_ドーナツ.kml"), "2次_車30分")
to_kml(ring3, os.path.join(donuts_dir, "3次_車45分_ドーナツ.kml"), "3次_車45分")

# Zip（通常版/ドーナツ版）→ 自動DL
def zip_kml_only(zip_basename, files):
    tmp = f"{zip_basename}_tmp"; os.makedirs(tmp, exist_ok=True)
    for k in files: shutil.copy2(k, os.path.join(tmp, os.path.basename(k)))
    shutil.make_archive(zip_basename, 'zip', tmp); shutil.rmtree(tmp)

def zip_kml_folder(kml_folder, zip_basename):
    if not os.path.isdir(kml_folder): return None
    shutil.make_archive(zip_basename, 'zip', kml_folder)
    return f"{zip_basename}.zip"

zip_normal = f"{VENUE_SLUG}_KML_通常版_{ts}"
zip_donut  = f"{VENUE_SLUG}_KML_ドーナツ_{ts}"
zip_kml_only(zip_normal, kml_paths + [center_kml])
donut_zip_path = zip_kml_folder(donuts_dir, zip_donut)

print("=== 出力（GeoJSON）==="); [print(p) for p in geojson_paths]
print("\n=== 出力（KML）==="); [print(p) for p in kml_paths]
print("\n中心点（KML）:", center_kml)
print("\n=== ドーナツ（KML）==="); [print(os.path.join(donuts_dir, f)) for f in os.listdir(donuts_dir) if f.endswith(".kml")]
import os
print("\n=== Zip 出力 ===")
print("通常版KML（Zip）:", os.path.abspath(f"{zip_normal}.zip"))
if donut_zip_path: print("ドーナツ版KML（Zip）:", os.path.abspath(donut_zip_path))

if AUTO_DOWNLOAD:
    files.download(f"{zip_normal}.zip")
    if donut_zip_path: files.download(donut_zip_path)

print("\n▼My Maps 取込手順：3次→2次→1次→中心点の順でKMLをレイヤに、色は3次=薄/2次=中/1次=濃")


ORS_API_KEY を貼り付けて Enter: eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImFjNTk2NzQ1ZjllZDRmODI5YmZlN2EzMjQyZWZhZjI0IiwiaCI6Im11cm11cjY0In0=
会場名（例: チトセピア）: 南丹
中心座標（Googleマップの「緯度, 経度」例: 32.7881, 129.8562）: 35.106713383123775, 135.47109917584214
Key preview: eyJv...In0=
会場名: 南丹 / 座標: lat=35.106713, lon=135.471099
=== 出力（GeoJSON）===
南丹_非円形到達圏_driving-car_20250913_2219/南丹_非円形到達圏_driving-car_20250913_2219_1次_車15分.geojson
南丹_非円形到達圏_driving-car_20250913_2219/南丹_非円形到達圏_driving-car_20250913_2219_2次_車30分.geojson
南丹_非円形到達圏_driving-car_20250913_2219/南丹_非円形到達圏_driving-car_20250913_2219_3次_車45分.geojson

=== 出力（KML）===
南丹_非円形到達圏_driving-car_20250913_2219/南丹_非円形到達圏_driving-car_20250913_2219_1次_車15分.kml
南丹_非円形到達圏_driving-car_20250913_2219/南丹_非円形到達圏_driving-car_20250913_2219_2次_車30分.kml
南丹_非円形到達圏_driving-car_20250913_2219/南丹_非円形到達圏_driving-car_20250913_2219_3次_車45分.kml

中心点（KML）: 南丹_非円形到達圏_driving-car_20250913_2219/南丹_非円形到達圏_driving-car_20250913_2219_中心点.kml

=== ドーナツ（KML）===
南丹_非円形到達圏_driving-car

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


▼My Maps 取込手順：3次→2次→1次→中心点の順でKMLをレイヤに、色は3次=薄/2次=中/1次=濃
